# 4D Shubert Function with AdaptivePrecision

This notebook demonstrates the use of AdaptivePrecision for 4D polynomial approximation using the Shubert function.

**Key Features:**
- Float64 evaluation for performance
- BigFloat expansion for accuracy
- Coefficient sparsity analysis
- Comparative analysis with Float64Precision

In [ ]:
include(joinpath(dirname(Base.find_package("Globtim")), "..", ".globtim", "notebook_setup.jl"))
include("../../test/adaptive_precision_4d_framework.jl")

## Setup: 4D Shubert Function Parameters

In [ ]:
# Constants and Parameters
const n, a, b = 4, 2, 1
const scale_factor = a / b   # Scaling factor
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # L2-norm tolerance

# Function and basic parameters
f = shubert_4d
d = 4  # Initial degree
SMPL = 50  # Number of samples (increased for better accuracy)
center = [0.0, 0.0, 0.0, 0.0]

println("4D Shubert Function Setup:")
println("  Dimension: $n")
println("  Initial degree: $d")
println("  Samples: $SMPL")
println("  Scale factor: $scale_factor")
println("  Center: $center")

## Comparison: Float64Precision vs AdaptivePrecision

In [ ]:
# Create test input
TR = TestInput(f,
                dim = n, 
                center = center,
                GN = SMPL,
                sample_range = scale_factor,
                degree_max = d + 4
                )

println("Test input created with $(TR.N) samples")

In [ ]:
# Construct polynomials with different precisions
println("Constructing polynomials...")

# Float64 precision (original approach)
println("\n🔍 Float64Precision:")
@time pol_float64 = Constructor(TR, d, basis=:chebyshev, precision=Float64Precision)
println("  L2 norm: $(pol_float64.nrm)")
println("  Coefficients: $(length(pol_float64.coeffs))")
println("  Coefficient type: $(eltype(pol_float64.coeffs))")

# Adaptive precision (new approach)
println("\n🚀 AdaptivePrecision:")
@time pol_adaptive = Constructor(TR, d, basis=:chebyshev, precision=AdaptivePrecision)
println("  L2 norm: $(pol_adaptive.nrm)")
println("  Coefficients: $(length(pol_adaptive.coeffs))")
println("  Coefficient type: $(eltype(pol_adaptive.coeffs))")
println("  Precision setting: $(pol_adaptive.precision)")

## Extended Precision Analysis

In [ ]:
# Convert to monomial basis for detailed analysis
@polyvar(x[1:n])

println("Converting to monomial basis...")

# Float64 monomial expansion
println("\n📊 Float64Precision Monomial Expansion:")
@time mono_float64 = to_exact_monomial_basis(pol_float64, variables=x)
coeffs_f64 = [coefficient(t) for t in terms(mono_float64)]
println("  Monomial terms: $(length(coeffs_f64))")
println("  Coefficient type: $(typeof(coeffs_f64[1]))")
if length(coeffs_f64) > 0
    coeff_mags_f64 = [abs(Float64(c)) for c in coeffs_f64]
    println("  Coefficient range: $(minimum(coeff_mags_f64[coeff_mags_f64 .> 1e-15])) to $(maximum(coeff_mags_f64))")
end

# AdaptivePrecision monomial expansion
println("\n🎯 AdaptivePrecision Monomial Expansion:")
@time mono_adaptive = to_exact_monomial_basis(pol_adaptive, variables=x)
coeffs_adaptive = [coefficient(t) for t in terms(mono_adaptive)]
println("  Monomial terms: $(length(coeffs_adaptive))")
println("  Coefficient type: $(typeof(coeffs_adaptive[1]))")
if length(coeffs_adaptive) > 0
    coeff_mags_adaptive = [abs(Float64(c)) for c in coeffs_adaptive]
    println("  Coefficient range: $(minimum(coeff_mags_adaptive[coeff_mags_adaptive .> 1e-15])) to $(maximum(coeff_mags_adaptive))")
    
    # Show sample BigFloat coefficients
    println("  Sample BigFloat coefficients:")
    for i in 1:min(3, length(coeffs_adaptive))
        println("    $i: $(coeffs_adaptive[i])")
    end
end

## Coefficient Distribution and Sparsity Analysis

In [ ]:
# Analyze coefficient distribution for AdaptivePrecision
if length(coeffs_adaptive) > 0
    println("📈 Coefficient Distribution Analysis:")
    
    analysis = analyze_coefficient_distribution(mono_adaptive)
    println("  Total terms: $(analysis.n_total)")
    println("  Dynamic range: $(analysis.dynamic_range)")
    println("  Max coefficient: $(analysis.max_coefficient)")
    println("  Min coefficient: $(analysis.min_coefficient)")
    println("  Suggested thresholds: $(analysis.suggested_thresholds)")
    
    # Test different truncation thresholds
    println("\n✂️  Sparsity Analysis:")
    thresholds = [1e-15, 1e-12, 1e-10, 1e-8, 1e-6]
    
    for threshold in thresholds
        truncated_poly, stats = truncate_polynomial_adaptive(mono_adaptive, threshold)
        println("  Threshold $(threshold): keep $(stats.n_kept)/$(stats.n_total) terms ($(round(stats.sparsity_ratio*100, digits=1))% sparse)")
    end
    
    # Use suggested threshold for detailed analysis
    if length(analysis.suggested_thresholds) > 0
        best_threshold = analysis.suggested_thresholds[1]
        truncated_poly, stats = truncate_polynomial_adaptive(mono_adaptive, best_threshold)
        
        println("\n🎯 Optimal Truncation (threshold=$(best_threshold)):")
        println("  Original terms: $(stats.n_total)")
        println("  Kept terms: $(stats.n_kept)")
        println("  Removed terms: $(stats.n_removed)")
        println("  Sparsity ratio: $(round(stats.sparsity_ratio*100, digits=1))%")
        println("  Largest removed coefficient: $(stats.largest_removed)")
        println("  Smallest kept coefficient: $(stats.smallest_kept)")
    end
end

## Critical Point Analysis with AdaptivePrecision

In [ ]:
# Solve for critical points using AdaptivePrecision polynomial
println("🔍 Critical Point Analysis with AdaptivePrecision:")

try
    # Use the truncated polynomial if available, otherwise use full polynomial
    poly_for_solving = if @isdefined(truncated_poly)
        println("Using truncated polynomial for critical point detection...")
        truncated_poly
    else
        println("Using full AdaptivePrecision polynomial for critical point detection...")
        mono_adaptive
    end
    
    # Convert to Float64 for HomotopyContinuation (required)
    # Note: This step converts BigFloat coefficients to Float64 for the solver
    poly_f64_for_solving = sum(Float64.(coefficient.(terms(poly_for_solving))) .* monomials(poly_for_solving))
    
    df_adaptive = solve_and_parse(pol_adaptive, x, f, TR)
    sort!(df_adaptive, :z, rev=true)
    
    println("✅ Critical points found: $(nrow(df_adaptive))")
    println("Top 5 critical points:")
    for i in 1:min(5, nrow(df_adaptive))
        row = df_adaptive[i, :]
        println("  $i: ($(round(row.x1, digits=4)), $(round(row.x2, digits=4)), $(round(row.x3, digits=4)), $(round(row.x4, digits=4))) → z=$(round(row.z, digits=4))")
    end
    
catch e
    println("❌ Error in critical point analysis: $e")
    println("This may be due to polynomial complexity or numerical issues.")
end

## Performance and Accuracy Summary

In [ ]:
println("📊 Performance and Accuracy Summary:")
println("=" ^ 50)

# L2 norm comparison
norm_diff = abs(pol_adaptive.nrm - pol_float64.nrm)
norm_improvement = pol_adaptive.nrm < pol_float64.nrm

println("🎯 Approximation Quality:")
println("  Float64Precision L2 norm: $(pol_float64.nrm)")
println("  AdaptivePrecision L2 norm: $(pol_adaptive.nrm)")
println("  Norm difference: $(norm_diff)")
println("  AdaptivePrecision improvement: $(norm_improvement ? "Yes" : "No")")

# Coefficient comparison
println("\n🔢 Coefficient Analysis:")
println("  Float64 monomial terms: $(length(coeffs_f64))")
println("  AdaptivePrecision monomial terms: $(length(coeffs_adaptive))")
println("  Float64 coefficient type: $(typeof(coeffs_f64[1]))")
println("  AdaptivePrecision coefficient type: $(typeof(coeffs_adaptive[1]))")

# Sparsity summary
if @isdefined(stats)
    println("\n✂️  Sparsity Benefits:")
    println("  Original terms: $(stats.n_total)")
    println("  After truncation: $(stats.n_kept)")
    println("  Sparsity achieved: $(round(stats.sparsity_ratio*100, digits=1))%")
    println("  Storage reduction: $(round((1-stats.sparsity_ratio)*100, digits=1))%")
end

println("\n🚀 Key AdaptivePrecision Benefits:")
println("  ✅ Extended precision coefficients (BigFloat)")
println("  ✅ Maintained Float64 evaluation performance")
println("  ✅ Smart coefficient truncation for sparsity")
println("  ✅ Seamless integration with existing workflows")
println("  ✅ Automatic precision management")

println("\n💡 Next Steps:")
println("  • Test with higher degrees (6, 8, 10)")
println("  • Experiment with different truncation thresholds")
println("  • Compare critical point detection accuracy")
println("  • Analyze performance with larger sample sizes")
println("  • Test with other 4D functions (Gaussian, sparse, etc.)")